In [1]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

C:\Users\david\AppData\Local\Temp\ipykernel_4808\541476175.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import data

In [2]:
labels = pd.read_csv("Data/medical_tc_labels.csv")
labels

,condition_label,condition_name
0,1,neoplasms
1,2,digestive system diseases
2,3,nervous system diseases
3,4,cardiovascular diseases
4,5,general pathological conditions


In [4]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv("Data/medical_tc_train.csv")
data.head()

,condition_label,medical_abstract
0,5,"Tissue changes around loose prostheses. A canine model to investigate the effects of an antiinflammatory agent. The aseptically loosened prosthesis provided a means for investigating the in vivo and in vitro activity of the cells associated with the loosening process in seven dogs. The cells were isolated and maintained in culture for sufficient periods of time so that their biologic activity could be studied as well as the effect of different agents added to the cells in vivo or in vitro. The biologic response as determined by interleukin-1 and prostaglandin E2 activity paralleled the roentgenographic appearance of loosening and the technetium images and observations made at the time of revision surgery. The correlation between clinical, roentgenographic, histologic, and biochemical loosening indicates that the canine model is suitable for investigating the mechanisms of prosthetic failure. A canine model permits the study of possible nonsurgical therapeutic interventions with the ultimate hope of stopping or slowing the loosening process."
1,1,"Neuropeptide Y and neuron-specific enolase levels in benign and malignant pheochromocytomas. Neuron-specific enolase (NSE) is the isoform of enolase, a glycolytic enzyme found in the neuroendocrine system. Neuropeptide Y (NPY) is a peptide recently discovered in the peripheral and central nervous systems. Serum NSE and plasma NPY levels have been reported to be increased in some patients with pheochromocytoma. The authors evaluated whether the measurement of these molecules could help to discriminate between benign and malignant forms of pheochromocytoma. The NSE levels were normal in all patients with benign pheochromocytoma (n = 13) and elevated in one half of those with malignant pheochromocytoma (n = 13). Plasma NPY levels were on the average significantly higher in the malignant (177.1 +/- 38.9 pmol/l, n = 16) than in the benign forms of the disease (15.7 +/- 389 pmol/l, n = 24). However, there was no difference in the percentage of patients with elevated NPY levels. These results show that determination of serum NSE may be useful for distinguishing between malignant and benign pheochromocytoma; the measurement of plasma NPY is not useful for differentiating the two kinds of tumors."
2,2,"Sexually transmitted diseases of the colon, rectum, and anus. The challenge of the nineties. During the past two decades, an explosive growth in both the prevalence and types of sexually transmitted diseases has occurred. Up to 55 percent of homosexual men with anorectal complaints have gonorrhea; 80 percent of the patients with syphilis are homosexuals. Chlamydia is found in 15 percent of asymptomatic homosexual men, and up to one third of homosexuals have active anorectal herpes simplex virus. In addition, a host of parasites, bacterial, viral, and protozoan are all rampant in the homosexual population. Furthermore, the global epidemic of AIDS has produced a plethora of colorectal manifestations. Acute cytomegalovirus ileocolitis is the most common indication for emergency abdominal surgery in the homosexual AIDS population. Along with cryptosporidia and isospora, the patient may present to the colorectal surgeon with bloody diarrhea and weight loss before the diagnosis of human immunodeficiency virus (HIV) disease. Other patients may present with colorectal Kaposi's sarcoma or anorectal lymphoma, and consequently will be found to have seropositivity for HIV. However, in addition to these protean manifestations, one third of patients with AIDS consult the colorectal surgeon with either condylomata acuminata, anorectal sepsis, or proctitis before the diagnosis of HIV disease. Although aggressive anorectal surgery is associated with reasonable surgical results in some asymptomatic HIV positive patients, the same procedures in AIDS (symptomatic HIV positive) patients will often be met with disastrous results. It is incumbent upon the surgeon, therefore, to r

Ci sono duplicati nel dataframe?

In [18]:
# Non esistono osservazioni in cui la coppia label-abstract è uguale
len(data[data.duplicated()])

0

In [11]:
# Esistono per duplicati degli abstract
duplicated = data[data['medical_abstract'].duplicated()]
len(duplicated)

2105

In [14]:
# Questo perché gli abstract possono avere più di una classe
for row in range(2105):
    abstract = duplicated.iloc[row]['medical_abstract']
    print(data[data['medical_abstract'] == abstract]['condition_label'])

177    4
505    5
Name: condition_label, dtype: int64
506     5
563     3
6376    4
Name: condition_label, dtype: int64
492     3
580     4
2265    1
Name: condition_label, dtype: int64
58     1
597    3
Name: condition_label, dtype: int64
145     3
708     4
4660    5
Name: condition_label, dtype: int64
62     5
726    2
Name: condition_label, dtype: int64
24      5
737     3
5171    2
Name: condition_label, dtype: int64
90     5
815    1
Name: condition_label, dtype: int64
808    5
892    2
Name: condition_label, dtype: int64
792    4
905    5
Name: condition_label, dtype: int64
229    5
906    4
Name: condition_label, dtype: int64
295    5
997    4
Name: condition_label, dtype: int64
284     5
1011    3
Name: condition_label, dtype: int64
334     1
1013    2
Name: condition_label, dtype: int64
728     3
1027    5
Name: condition_label, dtype: int64
117     3
1092    5
Name: condition_label, dtype: int64
382     1
1146    3
Name: condition_label, dtype: int64
555     2
1167    1
Name

# Costruzione del Dataset da utilizzare

In [23]:
# Costruiamo i set contenente tutti gli abstract (senza duplicati)
abst_set = set(data['medical_abstract'])

In [94]:
Df_abst_set = pd.DataFrame(abst_set, columns = ['medical_abstract'])

In [95]:
# Creiamo una copia
Df_abst = Df_abst_set.copy()
# Aggiungiamo al dataframe 5 colonne corrispondenti a ciascuna label
for label in range(1,6):
    Df_abst[label] = 0

In [96]:
pd.reset_option('display.max_colwidth')
Df_abst.head()

,medical_abstract,1,2,3,4,5
0,Primary neoplasms of the hollow organs of the ...,0,0,0,0,0
1,Combination therapy for anaplastic giant cell ...,0,0,0,0,0
2,Multiple gunshot wounds of the head: an anthro...,0,0,0,0,0
3,False aneurysm of the internal mammary artery....,0,0,0,0,0
4,Inflammatory nodular reactions after hepatitis...,0,0,0,0,0


In [97]:
for row in range(len(Df_abst_set)):
    abst = Df_abst_set.iloc[row]['medical_abstract']
    # Cerca tutte le osservazioni con il suddetto abstract ed estraine la condition_label
    label_list = list(data[data['medical_abstract'] == abst]['condition_label'])
    # Per ogni label nella lista, cambia il valore del dataframe in "1"
    for label in label_list:
        Df_abst.iloc[row, label] = 1

In [98]:
Df_abst

,medical_abstract,1,2,3,4,5
0,Primary neoplasms of the hollow organs of the ...,1,0,0,0,0
1,Combination therapy for anaplastic giant cell ...,1,0,0,1,0
2,Multiple gunshot wounds of the head: an anthro...,0,0,0,0,1
3,False aneurysm of the internal mammary artery....,0,0,0,1,0
4,Inflammatory nodular reactions after hepatitis...,0,0,0,0,1
...,...,...,...,...,...,...
9440,Differences in QRS configuration during unipol...,0,0,0,1,0
9441,Cytotoxic T cell clones isolated from ovarian ...,1,0,0,0,0
9442,Mononuclear phagocytes: a major population of ...,1,0,0,0,0
9443,Improved postoperative course after spinous pr...,0,0,0,0,1


## Quante osservazioni per ciascuna classe?

In [105]:
for i in range(1,6):
    print(str(i) + ': ' + str(sum(Df_abst.iloc[:,i])))

1: 2530
2: 1195
3: 1540
4: 2441
5: 3844


## Train-Test Split

In [106]:
train, test = train_test_split(Df_abst, test_size = 0.25, random_state = 1234)

In [107]:
for i in range(1,6):
    print(str(i) + ': ' + str(sum(train.iloc[:,i])))

1: 1874
2: 887
3: 1181
4: 1832
5: 2875


In [108]:
for i in range(1,6):
    print(str(i) + ': ' + str(sum(test.iloc[:,i])))

1: 656
2: 308
3: 359
4: 609
5: 969


In [ ]:
# Save No PreProcessing
train.to_csv('Data/No PreProcessing/train.csv')
test.to_csv('Data/No PreProcessing/test.csv')

# PreProcessing

## StopWords Removal

In [109]:
def preprocess(text):
    # Remove punctuation and other non-alphanumeric characters
    text =  re.sub('[^a-zA-Z]', ' ', text)
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    # Remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # Join the words back into a string
    return ' '.join(words)

In [110]:
train['medical_abstract'] = train['medical_abstract'].apply(lambda x: preprocess(x))
test['medical_abstract'] = test['medical_abstract'].apply(lambda x: preprocess(x))

In [111]:
# Save train and test data
train.to_csv('Data/StopWords/train.csv')
test.to_csv('Data/StopWords/test.csv')

## StopWords with Acronyms

In [2]:
train = pd.read_csv('Data/No PreProcessing/train.csv')
test = pd.read_csv('Data/No PreProcessing/test.csv')

In [28]:
def preprocessA(text):
    # Remove punctuation and other non-alphanumeric characters
    text =  re.sub('[^a-zA-Z]', ' ', text)
    # Set everything to lower except for Acronyms
    words = text.split(' ')
    text = ' '.join([x.lower() for x in words if re.match(r'[A-Z]?[a-z]+$', x)])
    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # Join the words back into a string
    return ' '.join(words)

In [29]:
train['medical_abstract'] = train['medical_abstract'].apply(lambda x: preprocessA(x))
test['medical_abstract'] = test['medical_abstract'].apply(lambda x: preprocessA(x))

In [32]:
# Save train and test data
train.to_csv('Data/StopWordsA/train.csv')
test.to_csv('Data/StopWordsA/test.csv')

## Lemmatization

In [2]:
train = pd.read_csv('Data/StopWords/train.csv')
test = pd.read_csv('Data/StopWords/test.csv')

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.stem import WordNetLemmatizer
wordlemmatizer = WordNetLemmatizer()

def lemmatizer(text):
    text = text.split(' ')
    lemm_text = ' '.join([wordlemmatizer.lemmatize(word) for word in text])
    return lemm_text

train['medical_abstract']= train['medical_abstract'].apply(lambda x: lemmatizer(x))
test['medical_abstract']= test['medical_abstract'].apply(lambda x: lemmatizer(x))

In [5]:
# Save train and test data
train.to_csv('Data/StopWords + Lemm/train.csv')
test.to_csv('Data/StopWords + Lemm/test.csv')